<img src="img/header1.png"></img>

<h1 style="text-align: left;">ToT: PySpark Basics to Inter./Adv. in ± 15 Minutes</h1>
By: Imam Cholissodin | Dosen Fakultas Ilmu Komputer (FILKOM), Universitas Brawijaya (UB)

# **PySpark Streaming**: getTweetStreaming untuk Analisis Tren secara Real Time

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-stream").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

sc = spark.sparkContext

In [3]:
# import sparkCOntext & StreamingContext dari Pyspark Lib.
from __future__ import print_function
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [4]:
#sc = SparkContext(appName="pyspark-stream")
sc.setLogLevel('ERROR')
# ssc = spark streaming context dengan interval waktu proses 10 detik
ssc = StreamingContext(sc, 10)

In [5]:
# Konek ke socet broker menggunakan ssc
socket_stream = ssc.socketTextStream("54.208.1.149", 8501) # set for client
# socket_stream = ssc.socketTextStream("54.208.1.149", 9881) # set for client
# socket_stream.pprint()

In [6]:
# set Window length, artinya semua proses analisis akan dibatasi 
# dalam waktu 60 detik
lines = socket_stream.window(60)
# lines.pprint()

## Proses Stream
1. Menerima text Tweet, lalu simpan dalam lines, Input DStream
2. Split text menjadi term-term, Apply transformation on DStream : flatMap
3. Filter semua terms dengan hashtag(#), transformation : filter
4. Konversikan term menjadi lowercase, transformation : map
5. Map tiap tag set menjadi seperti (term, 1), transformation : map
6. Lalu Reduce & Hitung frek. dari tiap hashtag, reduceByKey hashtags = output DSStream

In [7]:
hashtags = lines.flatMap(lambda text: text.split(" ")).\
filter(lambda word:word.lower().startswith("#")).\
map(lambda word: (word.lower(),1)).\
reduceByKey(lambda a,b:a+b)

# hashtags.pprint()

In [8]:
# Sort hashtag base frek secara descending
author_counts_sorted_dstream = hashtags.transform(lambda foo:foo.sortBy(lambda x:x[0].lower()).\
                                                  sortBy(lambda x:x[1], ascending=False))

In [9]:
author_counts_sorted_dstream.pprint()

## Start Spark Streaming
kode Spark streaming tidak akan dijalankan sampai ssc.start() di run.
Dimana ssc.start akan membuat lineage & DAG dan melakukan lazy evaluation 
dan mulai menjalankan keseluruhan kode secara sekuen.

In [10]:
ssc.start()

In [ ]:
# awaitTermination() untuk stop ssc
# ketika kita kill cell ini, maka python akan mengirim signal
# ke awaitTermination() to stop job Spark Streaming
ssc.awaitTermination()

-------------------------------------------
Time: 2020-10-20 16:28:10
-------------------------------------------
('#trump', 3)
('#foxnews', 2)
('#america', 1)
('#americaneedspennsylvania', 1)
('#australia', 1)
('#aznewsmediart', 1)
('#bannon:', 1)
('#bidenharris', 1)
('#biden’s', 1)
('#california', 1)
...

-------------------------------------------
Time: 2020-10-20 16:28:20
-------------------------------------------
('#trump', 3)
('#americaneedspennsylvania', 2)
('#foxnews', 2)
('#trump2020', 2)
('#vote', 2)
('#2020trumplandslide', 1)
('#50cent', 1)
('#america', 1)
('#americafirst', 1)
('#americalast', 1)
...

-------------------------------------------
Time: 2020-10-20 16:28:30
-------------------------------------------
('#trump', 5)
('#trump2020', 3)
('#vote', 3)
('#americaneedspennsylvania', 2)
('#bidencrimefa…rt', 2)
('#covid19,', 2)
('#foxnews', 2)
('#hunterbiden', 2)
('#iran', 2)
('#votelikeyourlifedependsonit', 2)
...

-------------------------------------------
Time: 2020-1